# Polynomial interpolation: introduction

This notebook is based on Chapters 6 and 11 of 

<a id="thebook"></a>

> Süli, Endre and Mayers, David F. _An introduction to numerical analysis_. Cambridge University Press, Cambridge, 2003.
<https://doi.org/10.1017/CBO9780511801181> (ebook in [Helka](https://helka.helsinki.fi/permalink/358UOH_INST/1h3k2rg/alma9926836783506253))


We consider the interpolation of a given data set by the polynomial of lowest possible degree that passes through the points of the dataset. Polynomial interpolation, and related techniques such as [Bézier curves](https://en.wikipedia.org/wiki/B%C3%A9zier_curve), can be used to approximate complicated curves given a few points, see the [Utah teapot](https://en.wikipedia.org/wiki/Utah_teapot) for an iconic example. For our purposes, polynomial interpolation forms the basis for algorithms in numerical integration and numerical ordinary differential equations.

# Lagrange interpolation

Let $n \ge 0$ be an integer, define

$$
\mathbb P_n = \{p : \mathbb R \to \mathbb R : \text{$p$ is a polynomial of degree $\le n$} \},
$$

and consider the problem

> Let $x_i, y_i \in \mathbb R$, $i=0,\dots,n$, and suppose that $x_i \ne x_j$ for $i \ne j$.
>
> Find $p \in \mathbb P_n$ such that $p(x_i) = y_i$ for all $i=0,\dots,n$.

<div style="padding:25px; border: 2px solid gray;">

## Theorem: Lagrange interpolation
For integer $n \ge 1$ and distinct $x_i \in \mathbb R$, $i=0,\dots,n$, the polynomials

\begin{align*}
L_k(x) = \prod_{i=0, i \ne k}^n \frac{x-x_i}{x_k-x_i}, \qquad k=0,\dots,n,
\end{align*}

satisfy $L_k(x_i) = \delta_{ik}$ for all $i,k = 0,\dots,n$. 
Moreover, for any $y_i \in \mathbb R$, $i = 0,\dots,n$,

\begin{align*}
p(x) = \sum_{k=0}^n y_k L_k(x)
\end{align*}

satisfies $p \in \mathbb P_n$ and $p(x_i) = y_i$ for $i = 0,\dots,n$. Also, if $q \in \mathbb P_n$ and $q(x_i) = y_i$ for $i = 0,\dots,n$, then $q = p$.
Setting $L_0 = 1$, the same holds also for $n=0$. 
</div>

For a proof, see Lemma 6.1 and Theorem 6.1 in [the book](#thebook).

Let $k \ge 0$ be an interger and let $a < b$. We write $C^k(a,b)$ for the set of real valued functions that are continuous on $[a,b]$ and have continuous derivatives up to order $k$ on $[a,b]$.

<div style="padding:25px; border: 2px solid gray;">


## Theorem: error in Lagrange interpolation
Let $x_0, \dots, x_n \in [a,b]$ be distinct, let $f \in C^{n+1}(a,b)$, and set

\begin{align*}
p(x) = \sum_{k=0}^n f(x_k) L_k(x).
\end{align*}

Then for all $x \in [a,b]$ there is $\xi \in (a,b)$ such that 

\begin{align*}
f(x) - p(x) = \frac{f^{(n+1)}(\xi)}{(n+1)!} \prod_{i=0}^n (x-x_i).
\end{align*}
</div>


For a proof, see Theorem 6.2 in [the book](#thebook).

# Runge phenomenon

Let $a = -5$, $b = 5$ and consider the Lagrange interpolation polynomial $p$ of the function 

$$
f(x) = \frac{1}{1+x^2}
$$

with equally spaced points on $[a,b]$ 

$$
x_i = a + \frac{i(b-a)}{n}, \qquad i = 0,\dots,n.
$$

Let us find the maximum of $|f - p|$ on $[a,b]$. The maximum is at a critical point of $p - f$ (this sign is more convenient) or at one of the end points $a$ and $b$. A critical point $x$ satisfies $p'(x) - f'(x) = 0$ or equivalently the polynomial equation

$$
(1 + x^2)^2 p'(x) + 2x = 0.
$$

Let's solve this equation using the polynomial root finding algorithm of NumPy.

In [ ]:
import numpy as np
import scipy.interpolate as interp

def f(x):
    return 1/(1 + x**2)

def p(n, fun = f):
    '''Lagrange interpolation polynomial of f on [-5,5]'''
    xs = np.linspace(-5, 5, n+1)
    ys = fun(xs)
    return interp.lagrange(xs, ys)    

In [ ]:
# Let's solve critical points from the equation q p' + r = 0, where
# r = 2 x
# q = (1+x^2)^2 = x^4 + 2 x^2 + 1

def crit(p):
    '''Critical points of p - f on [-5, 5]'''
    r = np.poly1d([2, 0]) # 2x
    q = np.poly1d([1, 0, 2, 0, 1]) # x^4 + 2 x^2 + 1
    # The left-hand side of the equation, i.e. q p' + r
    lhs = np.polyadd(np.polymul(q, np.polyder(p)), r)
    zs = np.roots(lhs)
    # Select the real roots 
    ixs = np.abs(np.imag(zs)) < np.finfo(float).eps
    xs = np.real(zs[ixs])
    # Select the roots in (-5,5)
    ixs = np.abs(xs) < 5
    return xs[ixs]

Consider the $\| \cdot \|_\infty$-error as number of nodes increases.

In [ ]:
def dist(p):
    '''Maximum of |f-p| on [-5,5]'''
    xs = crit(p)
    # Append the end points
    xs = np.append(xs, [-5, 5])
    # polyval evaluates a polynomial using Horner's method 
    return np.max(np.abs(f(xs) - np.polyval(p, xs)))

ns = [2**n for n in range(1,6)]
ds = [dist(p(n)) for n in ns]
import pandas as pd
df = pd.DataFrame(ds)
df.columns = ['Max error']
df.index = ns
df.index.name = 'n'
df.style.format('{:.2f}')

In [ ]:
# Plot f and p with n = 10
import matplotlib.pyplot as plt
xs = np.linspace(-5, 5, 100)
plt.plot(xs, f(xs), 'b') # f(x) in blue
plt.plot(xs, np.polyval(p(10), xs), 'r:'); # p_10(x) in red
plt.plot(xs, np.polyval(p(12), xs), 'g:'); # p_12(x) in green

Both $f$ and $p$ extend to the complex plane, however, $f$ is singular at $z = \pm \imath$. (Here $\imath = \sqrt{-1}$.) Let $C$ be a positively oriented simple closed curve in the complex plane and suppose that the line segment $[-5, 5]$ is inside $C$ and that $\pm \imath$ are both outside $C$. Let $x \in [-5,5]$ be distinct from $x_i$, $i=0,\dots,n$, and define 

$$
g(z) = \frac{f(z)}{z - x} \prod_{i=0}^n \frac{x - x_i}{z - x_i}.
$$

Then the [residue theorem](https://en.wikipedia.org/wiki/Residue_theorem) implies that 

$$
\frac{1}{2 \pi \imath} \oint_C g(z) dz = f(x) -  \sum_{k=0}^n f(x_k) L_k(x) = f(x) - p(x).
$$

The curve $C$ must intersect the line segment $\{\imath y : |y| < 1 \}$ on the imaginary axis since $\pm \imath$ are outside $C$. Let $z$ be a point in the intersection and let $x$ be close to $-5$. In general, the function $g$ is not small near $z$ since $|x - x_i|$ is larger than $|z - x_i|$ for $x_i$ near $5$.

### Example: global polynomials are bad with corners


In [ ]:
def tent(x):
    '''Tent function: 
    f(x) =  1 - |x| if |x| < 1,
            0       otherwise'''
    
    return 1 - np.fmin(np.abs(x), 1)

xs = np.linspace(-2,2,100)

plt.plot(xs, tent(xs), 'b')
plt.plot(xs, np.polyval(p(8, tent), xs), 'r:');
plt.plot(xs, np.polyval(p(16, tent), xs), 'g:');

# Linear interpolating splines

Instead of using a global polynomial approximation like Lagrange interpolation on a large interval, it is often better to divide the interval into small subintervals and look for a piecewise polynomial approximation. 

Let $a < b$ and let $m \ge 1$ be an integer. Let 

$$
a = x_0 < x_1 < \dots < x_m = b.
$$ 

The _linear spline_ interpolating $f \in C(a,b)$ is 

$$
s(x) = \frac{x_{i} - x}{x_i - x_{i-1}} f(x_{i-1}) + \frac{x - x_{i-1}}{x_i - x_{i-1}} f(x_i), \qquad x \in [x_{i-1}, x_i], \quad i = 1,\dots,m.
$$

We see that $s(x_i) = f(x_i)$ for all $i=0,\dots,m$, $s$ is continuous, and that $s$ a polynomial of first order on each subinterval.
Also, if $m = 1$, then $s \in \mathbb P_1$ is the Lagrange interpolation polynomial of $f$ with $x_0 = a$ and $x_1 = b$.

We write for $f \in C(a,b)$ 

$$
\|f\|_\infty = \max_{x \in [a,b]} |f(x)|.
$$

<div style="padding:25px; border: 2px solid gray;">

## Theorem: linear interpolation error
Let $f \in C^2(a,b)$ and let $s$ be the linear spline interpolating $f$ at 

\begin{align*}
a = x_0 < x_1 < \dots < x_m = b.
\end{align*}

Write $h = \max_{i=1,\dots,m} |x_i - x_{i-1}|$. Then 
\begin{align*}
\|f - s\|_\infty \le \frac12 \|h^2 f''\|_\infty.
\end{align*}
</div>

For a proof, with the sharp constant 1/8, see Theorem 11.1 in [the book](#thebook).

In [ ]:
# Linear spline interpolation of exp(-3x) on [0,1]
def f(x):
    return np.exp(-3*x)

xs = np.linspace(0,1, 4)
ys = f(xs)

s = interp.make_interp_spline(xs, ys, k=1)

xs_plot = np.linspace(0,1, 100)
plt.plot(xs_plot, f(xs_plot), 'b')
plt.plot(xs_plot, s(xs_plot), 'r')
plt.xticks(xs);
plt.grid(axis='x')

## Example: Vector graphics and Potrace

Images with fixed resolution (i.e. size in pixels) are called [raster images](https://en.wikipedia.org/wiki/Raster_graphics). They are easy to produce (both digitally and via physical device like a digital camera) and since _most_ of the screens we use are made of pixels in a grid, they are the natural format to present images (and by extension videos, animations etc.). However raster images do not work well with transformations like rotations and scaling. If we want to make the image very small and than later scale it back up, either the outcome is very bad or we have to store the original image which takes up a lot of space. File formats like `png`, `jpeg`, `tiff`, `gif`, `heif` and `bmp` are all rasterized.

Some of them use clever compression though and plenty of interesting mathematics can be done here using methods of [signal processing](https://en.wikipedia.org/wiki/Signal_processing) and linear algebra. For courses, see: [Mathematics of Photographs](https://studies.helsinki.fi/courses/course-unit/otm-8808bd27-99f1-47be-808f-f5038ac3ebbe/MAT21022), [Applications in matrices](https://studies.helsinki.fi/courses/course-unit/hy-CU-117377795-2021-08-01/MAT21019) and [Introduction to wavelets](https://studies.helsinki.fi/courses/course-unit/hy-CU-117630776-2021-08-01?cpId=hy-lv-73).

A Fundamentally different approach is given by [vector graphics](https://en.wikipedia.org/wiki/Vector_graphics) and the idea is very similar to polynomial interpolation: instead of storing the values in a long sequence of evenly spaced samples, we store the important nodes and interpolate between them with a polynomial (such as spline or Bézier curve). Different translations just move these nodes and the interpolation can be carried out as before at any desired resolution. File formats like `pdf`, `svg`, `eps` are used and different GIS and CAD softwares for example make heavy use of the methods. However often in practice this requires exporting raster graphics to vector format. [Potrace](https://potrace.sourceforge.net/) by Peter Selinger is an open source software that does this and the basic principle is [suprisingly simple](https://potrace.sourceforge.net/potrace.pdf).

<img src="https://potrace.sourceforge.net/img/head-orig3.png" alt="Original bitmap image" style="height: 256px; width:256px;"/>
<img src="https://potrace.sourceforge.net/img/head-smooth3.png" alt="Vector graphics image" style="height: 256px; width:256px;"/>  

*Left: original image. Right: Potrace output (converted to png).*

# Differentiation

Let 

$$
p(x) = \sum_{k=0}^n f(x_k) L_k(x)
$$

be the Lagrange interpolation polynomial of a function $f$,
with distinct 

$$x_0, \dots, x_n \in [a,b].
$$

Then $p'$ is an approximation of $f'$. We write $\partial$ for the derivative with respect to $x$.

<div style="padding:25px; border: 2px solid gray;">

## Theorem: error in differentiation
Let $f \in C^{n+1}(a,b)$, $n \ge 1$, and let $p \in \mathbb P_n$ be the Lagrange interpolation polynomial as above. Then there are $\eta_1,\dots,\eta_n \in (a,b)$ such that for all $x \in [a,b]$ there is $\xi \in (a,b)$ such that 
 
\begin{align*}
f'(x) - p'(x) = \frac{f^{(n+1)}(\xi)}{n!} \prod_{i=1}^n (x-\eta_i).
\end{align*}

Also, writing $h = b - a$, there holds

\begin{align*}
\|h\partial(f - p)\|_\infty \le \|(h\partial)^{n+1} f\|_\infty.
\end{align*}
</div>

_Proof_.
As $p$ is the Lagrange interpolation polynomial of $f$, there holds
\begin{align*}
f(x_i) - p(x_i) = 0, \quad i = 0, \dots, n.
\end{align*}
By the mean value theorem, there are $\eta_i \in (a,b)$, $i=1,\dots,n$, between consecutive points $x_i$, such that  
\begin{align*}
f'(\eta_i) - p'(\eta_i) = 0, \quad i = 1, \dots, n.
\end{align*}
Here $p' \in \mathbb P_{n-1}$ and $\eta_i$ are distinct. The uniqueness part of the Lagrange interpolation theorem implies that $p'$ is the Lagrange interpolation polynomial of $f'$ with respect to points $\eta_i$.
The claim follows from the error in Lagrange interpolation theorem.
$\blacksquare$

# First order finite differences

Let $a = t$ and $b = t + h$ for some $t \in \mathbb R$ and $h > 0$, and let $p \in \mathbb P_1$ be the Lagrange interpolation polynomial of a function $f \in C^2(a,b)$ with $x_0 = a$ and $x_1 = b$.
Then for $x \in [t, t + h]$

\begin{align*}
p(x) &= \frac{x - t}{h} f(t+h) + \frac{t + h - x}{h} f(t),
\\
p'(x) &= \frac{f(t+h) - f(t)}{h}.
\end{align*}

Note that $p'$ does not depend on $x$. It is called the [forward finite difference](https://en.wikipedia.org/wiki/Finite_difference) of $f$ at $t$. 

<div style="padding:25px; border: 2px solid gray;">

## Corollary: error in forward finite difference
Let $f \in C^2(a,b)$, $t \in [a,b)$, and $h > 0$. Suppose that $t+h \le b$. Then
\begin{align*}
\left|f'(t) - \frac{f(t+h) - f(t)}{h} \right| \le h \|f''\|_\infty.
\end{align*}
</div>

## Central finite difference

Let $a = t - h/2$ and $b = t + h/2$, and let $p \in \mathbb P_1$ be as above. Then

\begin{align*}
p(x) &= \frac{x - t + h/2}{h} f(t+h/2) + \frac{t + h/2 - x}{h} f(t-h/2),
\\
p'(x) &= \frac{f(t+h/2) - f(t - h/2)}{h},
\end{align*}

and $p'$ is called the central finite difference.

## Example: differentiation is nonetheless hard

Consider the central finite difference of a function $f$ at $t = 0$,
and suppose that $f(\pm h/2)$ is known only up to an additive error $\epsilon_\pm$.
Then we can not compute the correct finite difference, but only

$$
\frac{f(h/2) + \epsilon_+ - f(-h/2) - \epsilon_-}{h}
= \frac{f(h/2) - f(-h/2)}{h} + \frac{\epsilon_+ - \epsilon_-}{h}.
$$

This diverges as $h \to 0$ unless $\epsilon_+ = \epsilon_-$. In practice, errors $\epsilon_\pm$ of order of the machine epsilon are unavoidable, and choosing small $h$ will amplify these. 

Mitigating errors like this is discussed further in the inverse problems courses at UH.

# On the interpolation sub-package of SciPy

We have already seen [make_interp_spline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.make_interp_spline.html) that computes coefficients in a basis of splines so the resulting spline goes through points $(x_i, y_i)$, $i=0,\dots,n$. Basis splines (shortly B-splines) are discussed further in Section 11.6 of [the book](#thebook). We will next consider briefly cubic splines, see Sections 11.4-5. 

For more details on interpolation with SciPy see the [tutorial](https://docs.scipy.org/doc/scipy/tutorial/interpolate.html).

## Example: cubic splines

Let $a = x_0 < x_1 < \dots < x_m = b$. A function $s : [a,b] \to \mathbb R$
is a cubic spline interpolating a function $f \in C^1(a,b)$ if

1. $s|_{[x_{i-1},x_i]} \in \mathbb P_3$ for $i=1,\dots,m$,
2. $s(x_i) = f(x_i)$ for $i = 0,\dots,m$.

The _natural cubic spline_ $s$ is defined via the additional conditions

3. $s \in C^2(a,b)$,
4. $s''(x_0) = s''(x_m) = 0$.

The _Hermitian cubic spline_ $s$ is defined via the additional condition

3. $s'(x_i) = f'(x_i)$ for $i = 0,\dots,m$.

Let us compute interpolations of 

$$
f(x) = \frac{1}{1+x^2}
$$

on the interval $[0,5]$ by using the natural and Hermite cubic splines $s$ with 

$$
x_0 = 0, \quad x_1 = \frac53, \quad x_2 = \frac{10}3, \quad x_3 = 5.
$$

In [ ]:
def f(x):
    return 1/(1+x**2)
def fprime(x):
    '''The derivative of f'''
    return -2*x/(1+x**2)**2

xs = np.linspace(0, 5, 4)
ys = f(xs)
dys = fprime(xs)

In [ ]:
s_nat = interp.make_interp_spline(xs, ys, k=3, bc_type='natural')
s_her = interp.CubicHermiteSpline(xs, ys, dys)

xs_plot = np.linspace(0, 5)
plt.plot([0,5], [0,0], 'k'); # zero level 
plt.plot(xs_plot, f(xs_plot)-s_nat(xs_plot), 'r')
plt.plot(xs_plot, f(xs_plot)-s_her(xs_plot), 'g')
plt.xticks(xs)
plt.grid(axis='x')
plt.gca().set_xticklabels([f'$x_{n}$' for n in range(len(xs))]);

Clearly the green curve is closer to zero on $[x_1,x_3]$. That is, the Hermite cubic spline gives a better approximation than the natural one. On the other hand, its computation requires also the knowledge of $f'(x_i)$, $i=0,\dots,3$. (A similar plot can be found in [the book](#thebook), see Fig. 11.4.) 

## Chebyshev polynomials and nodes

We already know that the Lagrange polynomials give the unique polynomial for interpolating $n$ nodes (and Hermite polynomial simply include information about the derivative or slope at those nodes). We have also noticed that outside the nodes the approximation can be quite bad (e.g. the Runge phenomenaon). What about polynomials which match the the function $f$ in every point of the interval?

[Chebyshev polynomials](https://en.wikipedia.org/wiki/Chebyshev_polynomials) give the best $n$'th order polynomial approximation of $f(x) = x^{n+1}$ on the interval $[-1,1]$ in the sense of the $\| \cdot \|_\infty$-norm. They are covered in sections 8.4-5 of [the book](#thebook).

The $n$'th Chebyshev polynomial is $T_n(x) = \cos(n \cos^{-1}(x))$ but usually they are defined recursively since $T_0(x) = 1, \ T_1(x) = x$ and
$$ T_{n+1}(x) = 2x T_n(x) - T_{n-1}(x), \quad n = 1,2,..., \ x \in [-1,1].$$

In addition to the polynomials themselves, we also have that for $n \geq 1$, the zeros of $T_n$ are at
$$ x_j = \cos \left( \frac{(2j - 1)\pi}{2n} \right), \quad j = 1,2,...,n.$$
These points are called the *Chebyshev nodes*. They have a neat geometric setup on the circle.

In [ ]:
n = 9

def cheby_nodes(n):
    """Create n Chebyshev nodes on [-1,1]"""
    return np.cos(np.pi*np.linspace(1, 2*n - 1, n, endpoint=True)/(2*n))

chebys = cheby_nodes(n)
circle_points = np.linspace(1, 2*n - 1, n, endpoint=True)/(2*n)

xs = np.linspace(-1, 1, 100)
circle = np.sqrt(1 - xs**2)



fig, ax = plt.subplots()
ax.plot(xs,circle)
ax.grid(axis='x')
ax.scatter(chebys, np.zeros((n,1)), color='b')
for j in range(n):
    plt.plot((0,chebys[j]), (0,np.sin(np.pi*circle_points[j])), 'r--', linewidth=1)
    ax.annotate(f'$x_{j+1}$', (chebys[j] - 0.02, 0.02))
ax.scatter(chebys, np.sin(np.pi*circle_points), color='r');

But more importantly, the Chebyshev nodes give the smallest $\infty$-norm error for Lagrange polynomials!

Let's consider the Runge phenomenon again.

In [ ]:
def f(x):
    # This is the scaled variant for [-1,1]
    return 1/(5 + 25*x**2)

# Interpolation with Chebyshev nodes
ys = f(chebys)
chebyL = interp.lagrange(chebys, ys)   

# Interpolation with uniformly spaced nodes
xn = np.linspace(-1, 1, n+1)
L = interp.lagrange(xn, f(xn))

plt.plot(xs, f(xs), 'b', label='$f(x)$')
plt.plot(xs, chebyL(xs), 'g--', label='Chebyshev')
plt.plot(xs, L(xs), 'r--', label='Uniform')
plt.legend()